In [5]:
import numpy as np
import matplotlib.pyplot as plt

In [6]:
data = np.loadtxt("data.csv", delimiter=";", dtype=int)

In [25]:
years = data[:, 0]
population = data[:, 1]

1920

In [97]:
def poly1(t, j):
    return np.power(years[t],j, dtype=np.double)

def poly2(t,j):
    return np.power(years[t]-1900, j, dtype=np.double)

def poly3(t,j):
    return np.power(years[t]-1940,j, dtype=np.double)

def poly4(t,j):
    return np.power((years[t]-1940)/40,j, dtype=np.double)

In [98]:
matrix1 = np.fromfunction(poly1, (9,9), dtype=int)
matrix2 = np.fromfunction(poly2, (9,9), dtype=int)
matrix3 = np.fromfunction(poly3, (9,9), dtype=int)
matrix4 = np.fromfunction(poly4, (9,9), dtype=int)

In [99]:
cond_matrix1 = np.linalg.cond(matrix1)
cond_matrix2 = np.linalg.cond(matrix2)
cond_matrix3 = np.linalg.cond(matrix3)
cond_matrix4 = np.linalg.cond(matrix4)


In [100]:
print(cond_matrix1)
print(cond_matrix2)
print(cond_matrix3)
print(cond_matrix4)

4.428066803622041e+37
6211148482504961.0
9315536040586.037
1605.4437004786669


In [101]:
interpolation_factors = np.linalg.solve(matrix4, population.reshape(-1,1))

In [102]:
interpolation_factors

array([[ 1.32164569e+08],
       [ 4.61307656e+07],
       [ 1.02716315e+08],
       [ 1.82527130e+08],
       [-3.74614715e+08],
       [-3.42668456e+08],
       [ 6.06291250e+08],
       [ 1.89175576e+08],
       [-3.15180235e+08]])

In [103]:
xs = np.arange(1900,1991,1)
ys = np.polyval(interpolation_factors, xs)